In [1]:
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH17-LangGraph-Modules")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH17-LangGraph-Modules


In [2]:
from langgraph.checkpoint.memory import MemorySaver

# 메모리 저장소 생성
memory = MemorySaver()

In [3]:
from typing import Annotated
from typing_extensions import TypedDict
from langchain_openai import ChatOpenAI
from langchain_teddynote.tools.tavily import TavilySearch
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

# 1. 상태 정의
class State(TypedDict):
    # 메시지 목록 주석 추가
    messages: Annotated[list, add_messages]

# 2. 도구 정의 및 바인딩
tool = TavilySearch(max_results=3)
tools = [tool]

llm = ChatOpenAI(model="gpt-4o-mini")

llm_with_tools = llm.bind_tools(tools)

# 3. 노드 추가
def chatbot(state: State):
    return {"messages": llm_with_tools.invoke(state["messages"])}

graph_builder = StateGraph(State)

graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=tools)

graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition
)

# 4. 엣지 추가
graph_builder.add_edge("tools", "chatbot")

graph_builder.add_edge(START, "chatbot")

graph_builder.add_edge("chatbot", END)

In [4]:
# 그래프 빌더 컴파일
graph = graph_builder.compile(checkpointer=memory)

In [8]:
graph.get_graph().print_ascii()

        +-----------+         
        | __start__ |         
        +-----------+         
               *              
               *              
               *              
          +---------+         
          | chatbot |         
          +---------+         
          *         .         
        **           ..       
       *               .      
+---------+         +-------+ 
| __end__ |         | tools | 
+---------+         +-------+ 


# RunnableConfig 설정

In [6]:
from langchain_core.runnables import RunnableConfig

config = RunnableConfig(
    recursion_limit=10,  # 최대 10개의 노드까지 방문. 그 이상은 RecursionError 발생
    configurable={"thread_id": "1"},  # 스레드 ID 설정
)

In [14]:
# 첫 질문
question = (
    "내 이름은 `테디노트` 입니다. YouTube 채널을 운영하고 있어요. 만나서 반가워요"
)

for event in graph.stream({"messages": [("user", question)]}, config=config):
    for value in event.values():
        print(value["messages"])
        value["messages"].pretty_print()

content='안녕하세요, 테디노트님! 만나서 반갑습니다. YouTube 채널을 운영하신다니 멋지네요! 어떤 주제의 콘텐츠를 만들고 계신가요?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 602, 'total_tokens': 647, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_dbaca60df0', 'id': 'chatcmpl-BPNgOzUifM4EXVUmwAWcavzYyaSQm', 'finish_reason': 'stop', 'logprobs': None} id='run-4f7c35d0-10be-407e-bcca-1cf769475da1-0' usage_metadata={'input_tokens': 602, 'output_tokens': 45, 'total_tokens': 647, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
================================== Ai Message ==================================

안녕하세요, 테디노트님! 만나서 반갑습니다. YouTube 채널을 운영하신다니 멋지네요! 어떤 주제의 콘텐츠를 만들고 계신가요?


In [16]:
# 이어지는 질문
question = "내 이름이 뭐라고 했지?"

for event in graph.stream({"messages": [("user", question)]}, config=config):
    for value in event.values():
        value["messages"].pretty_print()

================================== Ai Message ==================================

당신의 이름은 "테디노트"입니다!


In [17]:
from langchain_core.runnables import RunnableConfig

question = "내 이름이 뭐라고 했지?"

config = RunnableConfig(
    recursion_limit=10,  # 최대 10개의 노드까지 방문. 그 이상은 RecursionError 발생
    configurable={"thread_id": "2"},  # 스레드 ID 설정
)

for event in graph.stream({"messages": [("user", question)]}, config=config):
    for value in event.values():
        value["messages"].pretty_print()

================================== Ai Message ==================================

당신의 이름을 알 수 없어요. 이름을 말씀해 주시면 기억해 둘 수 있습니다.


# 스냅샷: 저장된 State 확인

In [18]:
from langchain_core.runnables import RunnableConfig

config = RunnableConfig(
    configurable={"thread_id": "1"},  # 스레드 ID 설정
)
# 그래프 상태 스냅샷 생성
snapshot = graph.get_state(config)
snapshot.values["messages"]

[HumanMessage(content='내 이름은 `테디노트` 입니다. YouTube 채널을 운영하고 있어요. 만나서 반가워요', additional_kwargs={}, response_metadata={}, id='1b32d5d9-80c0-4826-bcff-8482ac4998b4'),
 AIMessage(content='안녕하세요, 테디노트님! 만나서 반갑습니다. YouTube 채널을 운영하고 계신다니 흥미롭네요! 어떤 콘텐츠를 주로 다루고 계신가요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 118, 'total_tokens': 165, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_dbaca60df0', 'id': 'chatcmpl-BPNdmlZ9nOcoZonZpGSi0jv9Vk5hV', 'finish_reason': 'stop', 'logprobs': None}, id='run-05ef8f28-1193-4a3b-ac1a-0cf61c08a1be-0', usage_metadata={'input_tokens': 118, 'output_tokens': 47, 'total_tokens': 165, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reas

In [19]:
# 설정된 config 정보
snapshot.config

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f0200a6-a152-6dee-8019-3d8f94658624'}}

In [20]:
# 저장된 값(values)
snapshot.values

{'messages': [HumanMessage(content='내 이름은 `테디노트` 입니다. YouTube 채널을 운영하고 있어요. 만나서 반가워요', additional_kwargs={}, response_metadata={}, id='1b32d5d9-80c0-4826-bcff-8482ac4998b4'),
  AIMessage(content='안녕하세요, 테디노트님! 만나서 반갑습니다. YouTube 채널을 운영하고 계신다니 흥미롭네요! 어떤 콘텐츠를 주로 다루고 계신가요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 118, 'total_tokens': 165, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_dbaca60df0', 'id': 'chatcmpl-BPNdmlZ9nOcoZonZpGSi0jv9Vk5hV', 'finish_reason': 'stop', 'logprobs': None}, id='run-05ef8f28-1193-4a3b-ac1a-0cf61c08a1be-0', usage_metadata={'input_tokens': 118, 'output_tokens': 47, 'total_tokens': 165, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'au

In [21]:
# 다음 노드
snapshot.next

()

In [23]:
snapshot.metadata["writes"]["chatbot"]["messages"]

AIMessage(content='당신의 이름은 "테디노트"입니다!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 690, 'total_tokens': 705, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_dbaca60df0', 'id': 'chatcmpl-BPNgf9xWNajLDnVO6tnMWW5n5jCQP', 'finish_reason': 'stop', 'logprobs': None}, id='run-0cfeec95-9540-43c6-8adc-e3cebf0548dc-0', usage_metadata={'input_tokens': 690, 'output_tokens': 15, 'total_tokens': 705, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [24]:
from langchain_teddynote.messages import display_message_tree

# 메타데이터(tree 형태로 출력)
display_message_tree(snapshot.metadata)

    source: "loop"
    writes:
        chatbot:
            messages:
                content: "당신의 이름은 "테디노트"입니다!"
                additional_kwargs: {"refusal": None}
                response_metadata:
                    token_usage:
                        completion_tokens: 15
                        prompt_tokens: 690
                        total_tokens: 705
                        completion_tokens_details: {"accepted_prediction_tokens": 0, "audio_tokens": 0, "reasoning_tokens": 0, "rejected_prediction_tokens": 0}
                        prompt_tokens_details: {"audio_tokens": 0, "cached_tokens": 0}
                    model_name: "gpt-4o-mini-2024-07-18"
                    system_fingerprint: "fp_dbaca60df0"
                    id: "chatcmpl-BPNgf9xWNajLDnVO6tnMWW5n5jCQP"
                    finish_reason: "stop"
                    logprobs: None
                type: "ai"
                name: None
                id: "run-0cfeec95-9540-43c6-8adc-e3cebf0548dc-0"
           